In [1]:
from striprtf.striprtf import rtf_to_text
from dotenv import load_dotenv
import pandas as pd
import openai
import json

In [2]:
load_dotenv()

True

In [3]:
def generate_prompt(text):
    return f"""
    Extract the
    first name as 'first_name',
    last name as 'last_name',
    SIGs as 'sigs',
    job title as 'job_title',
    company as 'company',
    chapter as 'chapter',
    work phone as 'phone',
    work street as 'street',
    work city as 'city',
    work state as 'state',
    work zip as 'zip',
    primary email as 'primary_email',
    secondary email as 'secondary_email',
    and LinkedIn address as 'linkedin_address'
    in JSON format.
    If there is no information for any given field, the value should be an empty string.
    The data is the following: {text}
"""

In [4]:
def extract_data(filename, output):
    with open(filename) as infile:
        content = infile.read()
    text = rtf_to_text(content)
    splitted_text = list(map(lambda string: string.strip(), text.split("------")))
    results = []
    for text in splitted_text:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that will help extraction information from text in JSON format."},
                    {"role": "user", "content": generate_prompt(text)},
                ]
            )
            json_response = response["choices"][0]["message"]["content"]
            results.append(json.loads(json_response))
        except Exception:
            pass
    processed_results = []
    for result in results:
        if isinstance(result, dict):
            processed_results.append(result)
        elif isinstance(result, list):
            processed_results.extend(result)
    df = pd.DataFrame(processed_results)
    df.to_excel(f"{output}.xlsx", index=False)

In [5]:
extract_data("data/Illinois-Controller List.rtf", "output/contacts-Illinois-Controller List.rtf")